Import librarries needed for LeNet implementation

In [1]:
# Import neccessary libraries
import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers.core import Dropout
from keras import backend as K


Using TensorFlow backend.


Import MNIST Data set available in keras library

In [2]:
from keras.datasets import mnist

batch_size = 128
num_classes = 10
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

# convert type to float32
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'No. of training images')
print(x_test.shape[0], 'No. of test images')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


11493376/11490434 [==============================] - 7s 1us/step
x_train shape: (60000, 28, 28, 1)
60000 No. of training images
10000 No. of test images


Build the model using keras

In [3]:
# Build the model using keras commands
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))


Compile the LeNet model

In [4]:
model.compile(loss=keras.losses.categorical_crossentropy, 
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

Train the model

In [5]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=12,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 124s 2ms/step - loss: 0.2568 - acc: 0.9207 - val_loss: 0.0552 - val_acc: 0.9821
Epoch 2/12
60000/60000 [==============================] - 117s 2ms/step - loss: 0.0867 - acc: 0.9738 - val_loss: 0.0399 - val_acc: 0.9865
Epoch 3/12
60000/60000 [==============================] - 120s 2ms/step - loss: 0.0636 - acc: 0.9810 - val_loss: 0.0314 - val_acc: 0.9888
Epoch 4/12
60000/60000 [==============================] - 121s 2ms/step - loss: 0.0548 - acc: 0.9834 - val_loss: 0.0303 - val_acc: 0.9900
Epoch 5/12
60000/60000 [==============================] - 118s 2ms/step - loss: 0.0472 - acc: 0.9860 - val_loss: 0.0366 - val_acc: 0.9880
Epoch 6/12
60000/60000 [==============================] - 121s 2ms/step - loss: 0.0420 - acc: 0.9874 - val_loss: 0.0294 - val_acc: 0.9905
Epoch 7/12
60000/60000 [==============================] - 116s 2ms/step - loss: 0.0372 - acc: 0.9891 - val_loss: 0.0254 - 

Evaluate the Model

In [9]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.02549291779803043
Test accuracy: 0.9905


Save the trained model as :
1. Weights : h5
2. Architecture : json

Load the saved files and evaluate the model 

In [10]:
from keras.models import model_from_json

# serialize model to JSON
lenet_model_json = model.to_json()
with open("model/lenet_model.json", "w") as json_file:
    json_file.write(lenet_model_json)

# serialize weights to HDF5
model.save_weights("model/lenet_model_weights.h5")
print("Saved model to disk")

# load the saved model
lenet_model_file = open('model/lenet_model.json', 'r')
lenet_model = lenet_model_file.read()
lenet_model_file.close()
lenet_model = model_from_json(lenet_model)

# load weights into new model
lenet_model.load_weights("model/lenet_model_weights.h5")
print("Loaded model from disk")
 
#evaluate the loaded model
lenet_model.compile(loss=keras.losses.categorical_crossentropy, 
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

score = lenet_model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Saved model to disk
Loaded model from disk
Test loss: 0.02549291779803043
Test accuracy: 0.9905


Save the trained model as:
1. Weights : h5
2. Architecture : YAML

Load the saved file and evaluate accuracy.  

In [11]:
from keras.models import model_from_yaml

# serialize model to JSON
lenet_model_yaml = model.to_yaml()
with open("model/lenet_model.yaml", "w") as yaml_file:
    yaml_file.write(lenet_model_yaml)

# serialize weights to HDF5
model.save_weights("model/lenet_model_weights.h5")
print("Saved model to disk")

# load the saved model
lenet_model_file = open('model/lenet_model.yaml', 'r')
lenet_model = lenet_model_file.read()
lenet_model_file.close()
lenet_model = model_from_yaml(lenet_model)

# load weights into new model
lenet_model.load_weights("model/lenet_model_weights.h5")
print("Loaded model from disk")
 
#evaluate the loaded model
lenet_model.compile(loss=keras.losses.categorical_crossentropy, 
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

score = lenet_model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Saved model to disk
Loaded model from disk
Test loss: 0.02549291779803043
Test accuracy: 0.9905
